# Some Title League
    by Gerald Liu and On Choi
     - CMSC 320 Fall 2019

<img src="https://media.comicbook.com/2019/04/riot-games-logo-1167492-1280x0.jpeg"  width="200" height="200"/><img src="https://i.pinimg.com/originals/4c/0b/51/4c0b5190cf49d2d3e16d8ab56c632b44.jpg" width="200" height="200"/>


## 1. Introduction

League of Legends (LoL) is a MOBA (Multiplayer Online Battle Arena) published by Riot Games in 2009 and is the [most popular online PC game](https://newzoo.com/insights/rankings/top-20-core-pc-games/) as of November 2019. The competitive scene or e-sports has been popular among players and the most recent international tornament had an estimated 99.6 million viewers and had a $2.5 million prize pool for over 100 teams accross 13 regions around the world [(source)](https://www.businessinsider.com/league-of-legends-world-championship-winner-funplus-phoenix-photos-2019-11). There is an estimated >100 million matches of league played per day with 8 million concurrent players at any given time as of 2017 and that number as only gone up. As a result, the LoL has generated a plethora of data over the years and much can be learned from any perspective and purpose. 


In our tutorial, we will looking at match and player data for the North American Server for the latest season as of December 2019. We will crawl the league API to gather and tidy useful data 

## 2. Required Libraries/Resources

 - To scrape League of Legends data, we need requets and beautifulsoup4
 - Pandas and Numpy will be used to transform and tidy our dataset
 - matplotlib and seaborn for data visualization
 - sklearn for predictions and machine learning
 
 
 - We need an API key to access the Riot API. We will load it from a hidden file called 'api_key.json'. Generate yours at https://developer.riotgames.com/. 

In [3]:
# Imports
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, validation_curve, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# API Key obtained from Riot Developer Portal (unique for account expires every 16 hours)
# However, you can apply for an permanent API key at https://developer.riotgames.com/app-type 
API_KEY = 'XXXXX-XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX'
# We have our API key loaded from a json file (can also be stored as an evironment variable if you would like)
with open('api_key.json') as f:
  API_KEY = json.load(f)['API_KEY']

# 3. Data Collection (API Crawling)

 - To gather match data, Riot provides an [API](https://developer.riotgames.com/apis) that has data for matches, players (summoners), champions (playable characters), ect.
 
 
 - However, Riot only provides match data from a given matchID which cannot be randomly gotten. Therefore, we need to start from a seeded playerID and crawl through that players' teammates and opponents matches to generate enough data to preform data analysis.
 
 
 - Our set seed will be my summoner ID "Fanw" on the North American server. 
 
 
 ### Using Riot's API
 
 - Riot has a [developer portal](https://developer.riotgames.com/) which you will need a Riot [account](https://developer.riotgames.com/login). 